# Reactive Programming

Reactive programming describes a design paradigm that relies on asynchronous programming logic to handle real-time updates to otherwise static content. A reactive application must be;
- **Responsive** takes action in a timely manner
- **Resilient** can handle failures and keep working 
- **Elastic** The system stays responsive under varying workload
- **Message Driven**  Reactive Systems rely on asynchronous message-passing

## Observer Pattern
The *Observer-Observable* pattern, which ensures that a group of objects gets alerted whenever another object's state changes.

- **Observable** notifies other objects of a change in the state
- **Observer** receive the notification and apply the suited work

>To receive the notifications the Observer must *subscribe* to the Observable

### Observables
Based on their behaviour, Observables can be distinguished into:

- *cold Observables*: they publish data only if at least one Observer is subscribed. The data is queued into a Data Stream (pull relation)
- *hot Observables*: they publish data independenlty of the number of Observers. As soon as an Observer is subscribed, it will start to receive data (from that point on).

> In the case of hot Observables data can be lost. This approach is typically used when data is irrilevant if not processed immediately

## ReactiveX
The most well-known implementation of the Reactive Programming paradigm is ReactiveX. It's built on the *Observer-Observable*.
The implemented library for python is the RxPy.
```
import rx
```

An Observable can be created to publish data from static data and dynamic data sources

```
observable = rx.from_list([1,2,3,4,5])
```

An Observer can be *subscribed* using the `subscribe` method:
```
subscribe(observer=None, on_error=None, on_completed=None, on_next=None, *, scheduler=None)
```
- `observer` The object that is to receive notifications.
- `on_error` Action to invoke upon exceptional termination of the observable sequence.
- `on_completed` Action to invoke upon graceful termination of the observable sequence.
- `on_next` Action to invoke for each element in the observable sequence.
- `scheduler` The default scheduler to use for this subscription.


In [12]:
import rx

def print_(val):
    print(f'(from method) Value received {val}\n',end='')


class DataObserver:
    def on_next(self, value):
        print(f'(from DataObserver) Value received {value}\n',end='')
    
    def on_error(self, error):
        print(f'(from DataObserver) Error {error}\n',end='')
    
    def on_completed(self):
        print(f'(from DataObserver) Completed!\n',end='')


observable = rx.from_list([1,2,3,4,5])
# use lambda
observable.subscribe(lambda val: print(f'(from lambda) Value received: {val}\n', end=''))

# use named function
observable.subscribe(print_)

# use Observable
observable.subscribe(DataObserver())

(from lambda) Value received: 1
(from lambda) Value received: 2
(from lambda) Value received: 3
(from lambda) Value received: 4
(from lambda) Value received: 5
(from method) Value received 1
(from method) Value received 2
(from method) Value received 3
(from method) Value received 4
(from method) Value received 5
(from DataObserver) Value received 1
(from DataObserver) Value received 2
(from DataObserver) Value received 3
(from DataObserver) Value received 4
(from DataObserver) Value received 5
(from DataObserver) Completed!


### Subjects
A *subject* is both an Observable and an Observer. Typically it is used to gather data from a source, modify it and redirect to the final Observers. Moreover, note for example in the previous case how each observer is served uniquely by the Observable; the Subject on the contrary is able to serve all the subscribed Observable before the next data item is ready.

In [23]:
import rx
from rx.subject import Subject
from datetime import datetime
from time import sleep
class TimeStampSubject(Subject):
    def on_next(self, value):
        now = datetime.now() # current date and time
        sleep(1)
        date_time = now.strftime("%m/%d/%Y, %H:%M:%S:%f")
        super().on_next((value, date_time))
    
    def on_error(self, error):
        print(f'(from DataObserver) Error {error}\n',end='')
    
    def on_completed(self):
        print(f'(from DataObserver) Completed!\n',end='')



def print_(val):
    val, date_time = val
    print(f'(from method) {date_time} - Value received: {val}\n',end='')



observable = rx.from_list([1,2,3,4,5])

subject = TimeStampSubject()
subject.subscribe(print_)
subject.subscribe(lambda val: print(f'(from lambda) {val[1]} - Value received: {val[0]}\n', end=''))

# The Observers are added to the subject before it is added to the Observable so that no data is lost
observable.subscribe(subject)

(from method) 10/20/2021, 19:28:18:918224 - Value received: 1
(from lambda) 10/20/2021, 19:28:18:918224 - Value received: 1
(from method) 10/20/2021, 19:28:19:932714 - Value received: 2
(from lambda) 10/20/2021, 19:28:19:932714 - Value received: 2
(from method) 10/20/2021, 19:28:20:942003 - Value received: 3
(from lambda) 10/20/2021, 19:28:20:942003 - Value received: 3
(from method) 10/20/2021, 19:28:21:946258 - Value received: 4
(from lambda) 10/20/2021, 19:28:21:946258 - Value received: 4
(from method) 10/20/2021, 19:28:22:953453 - Value received: 5
(from lambda) 10/20/2021, 19:28:22:953453 - Value received: 5
(from DataObserver) Completed!


### Concurrency
By default RxPy is single threaded. To do differently a scheduler must be provided when the subscribe method is called. There are different types of scheduler. From `rx.concurrency`: 

- ImmediateScheduler
- CurrentThreadScheduler
- TimeoutScheduler
- NewThreadScheduler
- ThreadPoolScheduler

`rx.concurrency.mainloopscheduler` defines other schedulers:
- `IOLoopScheduler`
- `PyGameScheduler`
- `WxScheduler`

## operators
It is possible to apply operators to the data stream. Operators are defined in the `rx.operators` module.
```
from rx import operators as op
```
To apply an operator it is necessary to create a pipe using the method `Observable.pipe()` and passing one or more operators.

### Creational Operators
Are used to cre